<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/2_1%EB%B2%88_Generating_CountVectorizer_Features_Bytime2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

In [0]:
Ability = train.loc[train['event']=='Ability',:].loc[train['time']<2,:]
import re
def SUB(x):
    result = re.sub(' ','',str(x)) + ','
    #result = re.sub('[[0-9]+]',    '',     result)
    result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
    #result = re.sub('Location:.+',    'Location:',     result)
    return result

Ability['event_contents'] = Ability.apply(lambda x: SUB(x['event_contents']), axis=1)
Ability['event_contents'].value_counts()

()-Attack;Location:(),                                              457150
()-TrainSCV,                                                        350376
()-MorphDrone,                                                      287254
()-TrainProbe,                                                      258765
()-TrainMarine,                                                     134801
                                                                     ...  
()-GravitonBeam;Target:ExtendingBridgeNEWide10Out[];Location:(),         1
()-KD8Charge;Target:SwarmHost[];Location:(),                             1
()-ScanMove;Target:SpineCrawler[];Location:(),                           1
()-BuildAssimilator;Target:StarportReactor[];Location:(),                1
()-Move;Target:StarportFlying[];Location:(),                             1
Name: event_contents, Length: 1232, dtype: int64

## 1. CountVevtorizer

In [0]:
def Ability_CountVectorizer_bytime(t):
    import pandas as pd
    import numpy as np
    import re
    def SUB(x):
        result = re.sub(' ','',str(x)) + ','
        #result = re.sub('[[0-9]+]',    '',     result)
        result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
        #result = re.sub('Location:.+',    'Location:',     result)
        return result

    train = pd.read_csv('train.csv')
    Ability2 = train.loc[train['event']=='Ability',:].loc[train['time']<t,:]

    Ability2['event_contents'] = Ability2.apply(lambda x: SUB(x['event_contents']), axis=1)
    #Ability2['event_contents'].value_counts()
    game_id = train.game_id.unique().tolist()
    columns = ['text0','text1','winner']
    mytrain = np.zeros(len(game_id)*len(columns))
    mytrain = mytrain.reshape(len(game_id),len(columns))
    mytrain = pd.DataFrame(mytrain, columns=columns)
    mytrain.index = game_id
    mytrain.winner = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
    mytrain.loc[:,'text0'] = Ability2.loc[Ability2['player']==0,:].loc[Ability2['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
    mytrain.loc[:,'text1'] = Ability2.loc[Ability2['player']==1,:].loc[Ability2['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
    del train
    del Ability2
    mytrain = mytrain.fillna('')

    test = pd.read_csv('test.csv')
    Ability2 = test.loc[test['event']=='Ability',:].loc[test['time']<t,:]

    Ability2['event_contents'] = Ability2.apply(lambda x: SUB(x['event_contents']), axis=1)
    #Ability2['event_contents'].value_counts()
    game_id = test.game_id.unique().tolist()
    columns = ['text0','text1']
    mytest = np.zeros(len(game_id)*len(columns))
    mytest = mytest.reshape(len(game_id),len(columns))
    mytest = pd.DataFrame(mytest, columns=columns)
    mytest.index = game_id

    mytest.loc[:,'text0'] = Ability2.loc[Ability2['player']==0,:].loc[Ability2['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
    mytest.loc[:,'text1'] = Ability2.loc[Ability2['player']==1,:].loc[Ability2['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
    del test
    del Ability2
    mytest = mytest.fillna('')

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import accuracy_score, roc_auc_score
    import pandas as pd
    import numpy as np
    import pickle
    from sklearn.model_selection import train_test_split
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.feature_extraction.text import CountVectorizer
    def tokenizer(text):
        return text.split(',')

    ########### 사용할 모수들 ###################
    min_df = 1
    ngram_range = (1,1)
    seed = 42

    ######################## 데이터로부터 단어가방 생성 ####################
    vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
    revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
    vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
    print(vector_reviews0.shape)

    vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
    revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
    vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
    print(vector_reviews1.shape)

    player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
    player0 = player0.add_prefix('player0_')
    player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
    player1 = player1.add_prefix('player1_')
    X = pd.concat([player0,player1],axis=1)
    y = mytrain.winner
    print(X.shape)
    del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
    del player0, player1


    ######################## Test data ####################
    vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
    revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
    vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
    print(vector_reviews0.shape)

    vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
    revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
    vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
    print(vector_reviews1.shape)

    player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
    player0 = player0.add_prefix('player0_')
    player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
    player1 = player1.add_prefix('player1_')

    del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

    X_test = pd.concat([player0,player1],axis=1)
    print(X_test.shape)

    del player0, player1

    ################ Train 이랑 Test에서 겹치는 column들만 남기기 ###########
    print(len(X.columns))
    print(len(X_test.columns))
    print(  len(    set(X.columns) & set(X_test.columns)   ) )

    use_col = list(    set(X.columns) & set(X_test.columns)   )

    X = X.loc[:,use_col]
    X_test = X_test.loc[:,use_col]

    X = X.add_prefix('Time_'+str(t) +'_')
    X_test = X_test.add_prefix('Time_'+str(t) +'_' )
    return X, X_test


## time < 2

In [0]:
t=2
X, X_test = Ability_CountVectorizer_bytime(t)

X.to_pickle('train_Ability'+str(t)+'.pkl')
X_test.to_pickle('test_Ability'+str(t)+'.pkl')

(38872, 134)
(38872, 130)
(38872, 264)
(16787, 113)
(16787, 112)
(16787, 225)
264
225
211


## time < 4

In [0]:
t=4
X, X_test = Ability_CountVectorizer_bytime(t)

X.to_pickle('train_Ability'+str(t)+'.pkl')
X_test.to_pickle('test_Ability'+str(t)+'.pkl')

(38872, 264)
(38872, 267)
(38872, 531)
(16787, 239)
(16787, 235)
(16787, 474)
531
474
455


## time < 6

In [0]:
t=6
X, X_test = Ability_CountVectorizer_bytime(t)

X.to_pickle('train_Ability'+str(t)+'.pkl')
X_test.to_pickle('test_Ability'+str(t)+'.pkl')

(38872, 392)
(38872, 397)
(38872, 789)
(16787, 357)
(16787, 367)
(16787, 724)
789
724
697


## time < 8

In [0]:
t=8
X, X_test = Ability_CountVectorizer_bytime(t)

X.to_pickle('train_Ability'+str(t)+'.pkl')
X_test.to_pickle('test_Ability'+str(t)+'.pkl')

(38872, 470)
(38872, 467)
(38872, 937)
(16787, 436)
(16787, 441)
(16787, 877)
937
877
857


## time<10

In [0]:
t=10
X, X_test = Ability_CountVectorizer_bytime(t)

X.to_pickle('train_Ability'+str(t)+'.pkl')
X_test.to_pickle('test_Ability'+str(t)+'.pkl')

(38872, 503)
(38872, 497)
(38872, 1000)
(16787, 475)
(16787, 473)
(16787, 948)
1000
948
931


## 모델링 시작

In [0]:
train_Ability2 = pd.read_pickle('train_Ability2.pkl')
train_Ability4 = pd.read_pickle('train_Ability4.pkl')
train_Ability6 = pd.read_pickle('train_Ability6.pkl')
train_Ability8 = pd.read_pickle('train_Ability8.pkl')
train_Ability10 = pd.read_pickle('train_Ability10.pkl')
train_Ability = pd.read_pickle('train_Ability.pkl')
X = pd.concat([train_Ability2,train_Ability4,train_Ability6,train_Ability8,train_Ability10,train_Ability],axis=1)
del train_Ability2,train_Ability4,train_Ability6,train_Ability8,train_Ability10,train_Ability
print(X.shape)

(38872, 4087)


In [0]:
test_Ability2 = pd.read_pickle('test_Ability2.pkl')
test_Ability4 = pd.read_pickle('test_Ability4.pkl')
test_Ability6 = pd.read_pickle('test_Ability6.pkl')
test_Ability8 = pd.read_pickle('test_Ability8.pkl')
test_Ability10 = pd.read_pickle('test_Ability10.pkl')
test_Ability = pd.read_pickle('test_Ability.pkl')
X_test = pd.concat([test_Ability2,test_Ability4,test_Ability6,test_Ability8,test_Ability10,test_Ability],axis=1)
del test_Ability2,test_Ability4,test_Ability6,test_Ability8,test_Ability10,test_Ability
print(X_test.shape)

(16787, 4087)


In [0]:
train = pd.read_csv('train.csv')
y = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
del train

In [0]:
seed = 42
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
#################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=5000, random_state=seed,n_jobs=-1).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포 #  ngram (1,1) : 0.6889
## 2씩 time구간 나눠서 : 0.6868
## n_estimator 5000 : 0.6887
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  ngram (1,1) : 0.6824
## 2씩 time구간 나눠서 : 0.6834

y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5000,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.6887
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, s